In [2]:
from fastbook import *


# 协同过滤 collaborative filtering

协同过滤是一种在推荐系统中常用的方法，通过分析用户和项目之间的关系，来发现其他用户对于这种项目的潜在的兴趣，比如说在B站里面，你看了很多二次元动画片，b站就会给你推荐很多其他也喜欢看二次元动画片的人也喜欢看的东西。

这次用的数据集是一个叫MovieLens的数据集，主要包含了电影的一些数据。

In [3]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [4]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


这个数据集中有用户的编号，电影的编号，用户对于电影的评分，还有个时间戳

## 目标函数的处理

接下来，我们需要将电影的元素用一个向量来表示，比如第一个参数表示多大程度上是科幻片，第二个是动作电影的程度，第三个是这个电影是不是老电影，用一个（-1,1）的域来表示。比如这个《星战最后的绝地武士》这个电影（名字我猜的，我没看过），就是典型的科幻动作电影，但是不是老电影

In [5]:
last_skywalker = np.array([0.98,0.9,-0.9])

相同的我们也可以将用户的属性来表示出来，比如这个用户比较喜欢动作和科幻电影，但是不喜欢老电影

In [6]:
user1 = np.array([0.9,0.8,-0.6])

我们把这两个向量相乘,这个结果就是用户对于这个电影的喜欢程度

PS:在pytorch和numpy中，点乘是*，叉乘是@

In [7]:
(user1*last_skywalker).sum()

2.1420000000000003

换个例子，这个卡萨布兰卡绝对不是一个科幻电影，动作成分也基本没有，但是是一个实打实的老电影，这样的电影碰上user1会如何呢？

In [8]:
casablanca = np.array([-0.99,-0.3,0.8])

In [9]:
(user1*casablanca).sum()

-1.611

结果上来讲是一个代表不喜欢的负值。

OK,将这个特例拓展到整个样本空间，这样的话我们就能得到一个所有用户对于所有电影的喜好矩阵了。

## 创建数据集

In [10]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [11]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


pd有个特点就是能够进行和sql一样的联表操作，这两个表都有带一个movie字段，所以能够进行联表操作.

创建数据集的时候我们需要创建一个协同的数据集，调用`CollabDataLoaders`这个类中的从dataframe创建数据集的函数`from_df`，数据来源是ratings，需要进行分析的item是title这一列的数据，如果这一列的字段名是item就不用单独设置了，如果数据集中的rating不是这个名字的话，也需要有个参数名字是rating_name来设置

In [12]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=6,num_workers=0)
dls.show_batch()

,user,title,rating
0,542,My Left Foot (1989),4
1,422,Event Horizon (1997),3
2,311,"African Queen, The (1951)",4
3,595,Face/Off (1997),4
4,617,Evil Dead II (1987),1
5,158,Jurassic Park (1993),5


In [13]:
n_users  = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

`n_users`和`n_movies`这两个变量是存放用户的数量和电影的数量的，同时`n_factors`说明有5个评判维度,用一张图说明一下

<img alt="Latent factors with crosstab" width="900" caption="Latent factors with crosstab" id="xtab_latent" src="images/att_00041.png">

这个橙色的框中就是用户有的五个偏好向量，同样淡蓝色的框中也是电影所有的偏好向量，将这两个矩阵相乘，得到的就是白色部分的用户对于该电影的喜好程度了.

但是这又引申出来一个问题：如何得到特定用户对于特定电影的喜好程度？ 这就需要在这样的表中使用索引的查找内容。深度学习中并不包含这样的内容，但是我们可以用矩阵的乘法来表示出来，要用到独热编码。

In [14]:
one_hot_3 = one_hot(3, n_users).float()     
one_hot_3

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,


将用户集中的所有用户都使用独热编码进行编码，比如例子里面的将第三个用户用独热编码编码后得出一个（1，944）大小的矩阵，然后矩阵的第三个元素的值为1，其余全为0，将其与用户喜好的矩阵进行相乘，就能单独得到第三行的用户喜好矩阵（5，944），而且除了第三行元素外其他元素全部为0,这种方式称为embedding。

In [15]:
user_factors.t()@one_hot_3

tensor([-0.4586, -0.9915, -0.4052, -0.3621, -0.5908])

In [16]:
x,y = dls.one_batch()
x.shape

torch.Size([6, 2])

In [17]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return (users * movies).sum(dim=1)

In [18]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [19]:
learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time
0,1.584202,1.539200,01:40
1,1.303441,1.517572,01:43
2,1.036151,1.223593,01:40
3,0.897574,0.980512,01:41
4,0.848717,0.925403,01:39


这样一个基本的模型就做好了，但是我们还是要考虑一个问题，如果有用户习惯性的给一个电影打高分or低分，碰上一个大家都觉得不错的电影，这样这个推荐算法就不是很有效果了，这时候我们要考虑到所谓的偏差。下面是改进后的模型。

In [20]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)              #用户的偏差
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)            #电影的偏差
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)

给每个电影和用户加一个偏差变量，这个user_bias和movie_bias就是偏差量，并嵌入到矩阵中，

In [21]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.960878,0.983803,01:55
1,0.869596,0.974227,02:16
2,0.774579,0.935645,01:49
3,0.493746,0.921011,01:54
4,0.376816,0.925238,01:59


效果不好，loss是下降的，原因是出现了过拟合。解决方式是使用正则化先处理数据。

## 正则化

要避免过拟合，一个办法就是减少模型中变量的数量，但是在实际的操作中很少有这么做，而是将变量变的很小，这里用到的技术是权重衰减（weight decay）也被叫做L2正则化，虽然有一点点不同，但是基本可以认为是一种方法。

``` python
loss_with_wd = loss + wd * (parameters**2).sum()
```

这里的wd就是所谓的权重缩减，最初我们的想法是减少变量的数量来达到降低模型复杂度的效果，但是这个问题是一个NP问题，也就是我们需要花很多时间去解决的问题，于是退而求其次，将模型中的变量尽量接近0，以达到减少变量的近似效果。

对于上式的参数求导后，就可以得到梯度的一个关系

``` python
parameters.grad += wd * 2 * parameters
```


在fastai中使用正则化的方式是在调用fit_one_cycle()函数中加入参数wd

In [22]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)                    #正则化大小为0.1

epoch,train_loss,valid_loss,time
0,1.232177,1.081926,01:59
1,1.000025,1.055134,01:50
2,1.008135,0.992293,01:46
3,1.002289,0.936471,01:48
4,0.922965,0.915843,01:42


In [23]:
movie_bias = learn.model.movie_bias.squeeze()
idxs = movie_bias.argsort()[:5]
[dls.classes['title'][i] for i in idxs]

ModuleAttributeError: 'Embedding' object has no attribute 'squeeze'